# Exploratory Data Analysis — AQI Predictor
**Location:** Hyderabad, Sindh, Pakistan  
**Data Source:** Open-Meteo Air Quality + Weather APIs  
**Target:** US AQI (US EPA Scale, 0–500)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta, timezone
import sys, os

sys.path.append(os.path.abspath('..'))
from src.database import get_db_client
import src.config as config

sns.set_theme(style='darkgrid')
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['figure.dpi'] = 100

## 1. Load Data from MongoDB

In [ ]:
db = get_db_client()
data = list(db[config.FEATURE_COLLECTION].find({}, {'_id': 0}).sort('time', 1))
df = pd.DataFrame(data)
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
df.sort_index(inplace=True)

print(f'Records: {len(df)}')
print(f'Date range: {df.index.min()} to {df.index.max()}')
print(f'Duration: {(df.index.max() - df.index.min()).days} days')
print(f'Columns: {list(df.columns)}')
df.head()

## 2. Data Overview & Missing Values

In [ ]:
print('=== Column Statistics ===')
for col in df.select_dtypes(include=[np.number]).columns:
    nulls = df[col].isnull().sum()
    print(f'  {col:20s} | nulls={nulls:4d} | range=[{df[col].min():.1f}, {df[col].max():.1f}] | mean={df[col].mean():.1f}')

# Check for time gaps
time_diffs = df.index.to_series().diff().dropna()
gaps = time_diffs[time_diffs > pd.Timedelta(hours=1)]
print(f'\nTime gaps > 1 hour: {len(gaps)}')
if len(gaps) > 0:
    for t, gap in gaps.items():
        print(f'  {t}: {gap}')

## 3. Target Variable (US AQI) Distribution

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df['us_aqi'].dropna(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df['us_aqi'].mean(), color='red', linestyle='--', label=f'Mean: {df["us_aqi"].mean():.1f}')
axes[0].axvline(df['us_aqi'].median(), color='orange', linestyle='--', label=f'Median: {df["us_aqi"].median():.1f}')
axes[0].set_xlabel('US AQI')
axes[0].set_ylabel('Frequency')
axes[0].set_title('AQI Distribution')
axes[0].legend()

# Box plot
axes[1].boxplot(df['us_aqi'].dropna(), vert=True)
axes[1].set_ylabel('US AQI')
axes[1].set_title('AQI Box Plot')

# AQI categories
cats = pd.cut(df['us_aqi'], bins=[0, 50, 100, 150, 200, 300, 500],
              labels=['Good', 'Moderate', 'USG', 'Unhealthy', 'Very Unhealthy', 'Hazardous'])
cat_counts = cats.value_counts().sort_index()
colors = ['#00e400', '#ffff00', '#ff7e00', '#ff0000', '#8f3f97', '#7e0023']
axes[2].bar(cat_counts.index, cat_counts.values, color=colors[:len(cat_counts)])
axes[2].set_ylabel('Hours')
axes[2].set_title('AQI Category Distribution')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print('\nAQI Category Breakdown:')
for cat, count in cat_counts.items():
    print(f'  {cat:20s}: {count:5d} ({count/len(df)*100:.1f}%)')

## 4. Time Series Plot

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.plot(df.index, df['us_aqi'], linewidth=0.5, alpha=0.8, color='steelblue')
ax.axhline(50, color='green', linestyle='--', alpha=0.5, label='Good/Moderate boundary')
ax.axhline(100, color='orange', linestyle='--', alpha=0.5, label='Moderate/USG boundary')
ax.axhline(150, color='red', linestyle='--', alpha=0.5, label='USG/Unhealthy boundary')
ax.set_xlabel('Date')
ax.set_ylabel('US AQI')
ax.set_title('US AQI Time Series — Hyderabad, Sindh')
ax.legend()
plt.tight_layout()
plt.show()

## 5. Hourly and Day-of-Week Patterns

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Hourly pattern
hourly = df.groupby(df.index.hour)['us_aqi'].mean()
axes[0].bar(hourly.index, hourly.values, color='steelblue', alpha=0.8)
axes[0].set_xlabel('Hour of Day')
axes[0].set_ylabel('Mean AQI')
axes[0].set_title('Average AQI by Hour of Day')
axes[0].set_xticks(range(0, 24))

peak_h = hourly.idxmax()
low_h = hourly.idxmin()
print(f'Peak AQI hour: {peak_h}:00 (AQI={hourly[peak_h]:.1f})')
print(f'Lowest AQI hour: {low_h}:00 (AQI={hourly[low_h]:.1f})')
print(f'Variation: {hourly.max() - hourly.min():.1f} AQI')

# Day of week pattern
dow = df.groupby(df.index.dayofweek)['us_aqi'].mean()
dow_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(dow_names, dow.values, color='coral', alpha=0.8)
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Mean AQI')
axes[1].set_title('Average AQI by Day of Week')

print(f'\nWeekday avg: {dow[:5].mean():.1f}')
print(f'Weekend avg: {dow[5:].mean():.1f}')

plt.tight_layout()
plt.show()

## 6. Autocorrelation Analysis
How well does past AQI predict future AQI?

In [ ]:
lags = [1, 3, 6, 12, 24, 48, 72]
autocorrs = []

aqi = df['us_aqi'].dropna()
for lag in lags:
    corr = aqi.corr(aqi.shift(lag))
    autocorrs.append(corr)
    print(f'  Lag {lag:3d}h: r = {corr:.3f}')

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar([str(l) for l in lags], autocorrs, color='steelblue', alpha=0.8)
ax.set_xlabel('Lag (hours)')
ax.set_ylabel('Autocorrelation (r)')
ax.set_title('AQI Autocorrelation at Different Lags')
ax.axhline(0.7, color='red', linestyle='--', alpha=0.5, label='r=0.7 threshold')
ax.legend()
plt.tight_layout()
plt.show()

print('\nKey finding: AQI is highly persistent (r=0.74 at 24h lag).')
print('This means lag features will be our strongest predictors.')

## 7. Weather vs AQI Correlations

In [ ]:
weather_cols = ['temp', 'humidity', 'wind_speed', 'rain']
pollutant_cols = ['pm10', 'no2', 'ozone']
all_cols = weather_cols + pollutant_cols + ['us_aqi']

corr_matrix = df[all_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, ax=ax,
            square=True, linewidths=0.5)
ax.set_title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

print('\nCorrelations with US AQI:')
aqi_corr = corr_matrix['us_aqi'].drop('us_aqi').sort_values(key=abs, ascending=False)
for col, corr in aqi_corr.items():
    direction = 'Higher value → LOWER AQI' if corr < 0 else 'Higher value → HIGHER AQI'
    print(f'  {col:15s}: r = {corr:+.3f}  ({direction})')

## 8. AQI by Temperature and Wind Speed
The two strongest weather predictors from the correlation analysis.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Temperature bins
df['temp_bin'] = pd.cut(df['temp'], bins=5)
temp_aqi = df.groupby('temp_bin')['us_aqi'].mean()
axes[0].bar(range(len(temp_aqi)), temp_aqi.values, color='tomato', alpha=0.8)
axes[0].set_xticks(range(len(temp_aqi)))
axes[0].set_xticklabels([f'{x.left:.0f}-{x.right:.0f}°C' for x in temp_aqi.index], rotation=45)
axes[0].set_ylabel('Mean AQI')
axes[0].set_title('Average AQI by Temperature Range')

# Wind speed bins
df['wind_bin'] = pd.cut(df['wind_speed'], bins=5)
wind_aqi = df.groupby('wind_bin')['us_aqi'].mean()
axes[1].bar(range(len(wind_aqi)), wind_aqi.values, color='skyblue', alpha=0.8)
axes[1].set_xticks(range(len(wind_aqi)))
axes[1].set_xticklabels([f'{x.left:.0f}-{x.right:.0f} km/h' for x in wind_aqi.index], rotation=45)
axes[1].set_ylabel('Mean AQI')
axes[1].set_title('Average AQI by Wind Speed Range')

plt.tight_layout()
plt.show()

print('\nKey finding: Cold + calm = worst AQI (inversions).')
print('Hot + windy = best AQI (mixing and dispersion).')

## 9. PM2.5 vs AQI Relationship
US AQI in Hyderabad is driven almost entirely by PM2.5.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
valid = df[['pm2_5', 'us_aqi']].dropna()
ax.scatter(valid['pm2_5'], valid['us_aqi'], alpha=0.1, s=5, color='steelblue')
ax.set_xlabel('PM2.5 (µg/m³)')
ax.set_ylabel('US AQI')
ax.set_title(f'PM2.5 vs US AQI (r = {valid["pm2_5"].corr(valid["us_aqi"]):.3f})')
plt.tight_layout()
plt.show()

print('PM2.5 is the dominant pollutant driving AQI in Hyderabad.')
print('This is why we EXCLUDE pm2_5 from model features — it would be data leakage.')

## 10. Feature Design Rationale

Based on the EDA findings, we designed **28 features** in 5 groups:

| Group | Features | Rationale |
|-------|----------|-----------|
| **Weather** | temp, humidity, wind_speed, rain, weather_code | wind (r=-0.40), temp (r=-0.39) are strong predictors |
| **Pollutants** | pm10, no2, ozone | pm10 (r=+0.45) strongest; available from forecast |
| **Cyclical Time** | hour_sin/cos, dow_sin/cos, month_sin/cos | Captures diurnal, weekly, seasonal cycles |
| **Explicit Time** | hour, is_night, is_morning_rush, etc. | Binary indicators for high-AQI periods |
| **Lag/Rolling** | us_aqi_lag_24h, rolling_mean_24h, rolling_std_24h | lag_24h has r=0.739 — strongest predictor |
| **Interactions** | wind_x_temp, humidity_x_temp, pm10_x_humidity | Capture non-linear relationships |

### Excluded from features:
- **us_aqi** — this is the target variable
- **pm2_5** — directly derived from us_aqi (data leakage)
- **city** — constant (always Hyderabad)

## 11. Open-Meteo Data Delay Analysis
Understanding the gap between real-time and available data.

In [ ]:
import requests

# Query Open-Meteo with forecast_days=0 to get ONLY real (non-forecast) data
url = 'https://air-quality-api.open-meteo.com/v1/air-quality'
params = {
    'latitude': config.LAT, 'longitude': config.LON,
    'hourly': ['us_aqi'],
    'past_days': 3, 'forecast_days': 0,
    'timezone': 'Asia/Karachi'
}

resp = requests.get(url, params=params, timeout=30).json()
times = resp['hourly']['time']
aqis = resp['hourly']['us_aqi']

# Find last non-null
for i in range(len(aqis)-1, -1, -1):
    if aqis[i] is not None:
        from datetime import datetime, timedelta, timezone as tz
        PKT = tz(timedelta(hours=5))
        now = datetime.now(PKT).replace(tzinfo=None)
        last = datetime.fromisoformat(times[i])
        delay = (now - last).total_seconds() / 3600
        print(f'Current time (PKT): {now.strftime("%Y-%m-%d %H:%M")}')
        print(f'Last real AQI: {aqis[i]} at {times[i]}')
        print(f'Delay: {delay:.1f} hours')
        print()
        print('The air quality data comes from CAMS (Copernicus) which updates daily.')
        print('Weather data (temp, wind) updates every 1-6 hours.')
        print('When using forecast_days>=1, today\'s hours are filled with CAMS model predictions.')
        break

## Summary of Key Findings

1. **AQI is predominantly Moderate** (51-100) with occasional USG days
2. **Strong autocorrelation** (r=0.74 at 24h lag) — yesterday's AQI is the best predictor
3. **Wind speed** (r=-0.40) and **temperature** (r=-0.39) are the strongest weather predictors
4. **PM10** (r=+0.45) is the strongest pollutant predictor (physically related to PM2.5)
5. **Mild diurnal pattern** (~3.4 AQI variation) — evening peak, afternoon low
6. **No weekday/weekend difference** — AQI driven by weather, not local traffic
7. **Cold + calm = worst AQI** (temperature inversions trap pollutants)
8. **Data is continuous** — no time gaps in the 189-day dataset